In [2]:
import os
%load_ext autoreload
%autoreload 2

In [3]:
cd ~/workspace/theislab/mubind/notebooks/batch

/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks/batch


In [17]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations' # '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import numpy as np

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda:0


In [18]:
df = mb.bindome.datasets.ProBound.ctcf(flank_length=0)
df = df.sort_values(1, ascending=False).reset_index(drop=True)
# data = df.head(1000)
# data = df.copy()

df.index = df['seq']
del df['seq']
df.index = df.index.astype(str).str[45:]

df = df[~df.index.duplicated(keep='first')]

n_sample = 1000
data = df.sample(min(df.shape[0], n_sample))

# data = mb.pp.sample_rounds(df, 2, 10000)

# remove as many nucleotides as posisble (faster training/convergence)


# data.index = data.index.astype(str).str[-15:]


In [19]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 5, 1

In [20]:
# TODO: skipped for now
n_rounds = 2

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, labels=[0, 1])
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=625,
                         shuffle=False)

loading object (# entries) 1000


In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# %lprun -f mb.tl.train_network mb.tl.train_network(model, train, device, next_optimiser, criterion, num_epochs=20, early_stopping=100, log_each=2, dirichlet_regularization=0, exp_max=40, verbose=0)

In [23]:
# %lprun -f mb.tl.train_iterative mb.tl.train_iterative(train, device, w=18, show_logo=False, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=[0.01, 0.01], weight_decay=[0.01, 0.001], ignore_kernel=ignore_kernel, num_epochs=2, early_stopping=100, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None)

In [24]:
pd.set_option('display.expand_frame_repr', False)


In [25]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 100
log_each = 10
n_kernels = 4
lr = [0.01] * n_kernels
wd = [0.01,] + [0.001] * (n_kernels - 1)
early_stopping = [5,] + [10] * (n_kernels - 1)

opt_kernel_shift = [0, 0, 1, 1]
opt_kernel_length = [0, 0, 1, 1]
use_mono = True
use_dinuc = False
dinuc_mode = 'local'

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()
model = mb.models.Multibind.make_model(train, n_kernels, criterion, kernels=[0, 2, 20, 20],
                                       use_dinuc=use_dinuc, dinuc_mode=dinuc_mode).cuda()


here... 2
Using device: cuda:0


In [26]:

model.optimize_neg_weight = False
model.optimize_kernel_rel = False
model.optimize_sym_weight = True

model, best_loss = model.optimize_iterative(train,
                                            show_logo=0,
                                            r2_per_epoch=True,
                                            opt_kernel_length=[0, 0, 1, 1],
                                            opt_kernel_shift=[0, 0, 1, 1],
                                            dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                            # lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel,
                                            # skip_kernels={2, 3},
                                            n_epochs=n_epochs,
                                            early_stopping=early_stopping, # optimiser=torch.optim.LBFGS,
                                            log_each=log_each,
                                            stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
print('##DONE....\n\n')



print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

# model = model_by_k



current kernels

Kernel to optimize 0

FREEZING KERNELS
optimizing feature type mono
setting grad status of kernel (mono, dinuc) at 0 to (1, 0)
setting grad status of kernel (mono, dinuc) at 1 to (0, 0)
setting grad status of kernel (mono, dinuc) at 2 to (0, 0)
setting grad status of kernel (mono, dinuc) at 3 to (0, 0)
kernels mask None
optimizer:  Adam 
criterion: PoissonLoss 
# epochs: 100 
early_stopping: 5
lr= 0.01, weight_decay= 0.001, dir weight= 0
Epoch: 11, Loss: 3.694774, R2: -1.12,  best epoch: 9,  secs per epoch: 0.044 s,  secs epoch*1k trials: 0.044s
Epoch: 21, Loss: 3.572593, R2: -1.12,  best epoch: 19,  secs per epoch: 0.036 s,  secs epoch*1k trials: 0.036s
Epoch: 31, Loss: 3.451047, R2: -1.11,  best epoch: 29,  secs per epoch: 0.032 s,  secs epoch*1k trials: 0.032s
Epoch: 41, Loss: 3.330453, R2: -1.09,  best epoch: 39,  secs per epoch: 0.030 s,  secs epoch*1k trials: 0.030s
Epoch: 51, Loss: 3.211268, R2: -1.07,  best epoch: 49,  secs per epoch: 0.029 s,  secs epoch*1k tr

In [27]:
# model = model_by_k
mb.tl.scores(model, train, by='batch')


{0: {'r2_counts': 0.12972004140730942,
  'r2_foldchange': 0.0812279453134318,
  'r2_enr': 0.0035840488337016696,
  'r2_fc': 0.08735740217328772,
  'pearson_foldchange': 0.2955628565521854}}

In [28]:
counts = mb.tl.predict(model, train)
counts

In [29]:
train.dataset.rounds.shape

(1000, 2)

In [30]:
counts.shape

NameError: name 'counts' is not defined

In [ ]:
rcParams['figure.figsize'] = 6, 6
rcParams['figure.dpi'] = 35

mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1')


In [ ]:
rcParams['figure.figsize'] = 6, 6
rcParams['figure.dpi'] = 80

mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1')


In [ ]:

model = model_by_k
mb.tl.scores(model, train)


In [ ]:
rcParams['figure.figsize'] = 6, 6
rcParams['figure.dpi'] = 80

mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1', k=9)


In [ ]:
rcParams['figure.figsize'] = 8, 4
rcParams['figure.dpi'] = 100
mb.pl.loss(model)
